In [4]:
! pip install finbert

ERROR: Could not find a version that satisfies the requirement finbert (from versions: none)
ERROR: No matching distribution found for finbert


In [5]:
! pip install pandas

In [6]:
! pip install torch torchvision torchaudio
! pip install yfinance
! pip install transformers pandas
! pip install torch torchvision
! pip install nltk
! pip install textblob
! pip install plotly
! pip install scikit-learn

In [7]:
# Đọc file để xem qua cấu trúc dữ liệu
file_path = 'vn_economic_newspaper.txt'

# Do file có thể khá lớn, chúng ta sẽ đọc một phần của file để hiểu cấu trúc dữ liệu
with open(file_path, 'r', encoding='utf-8') as file:
    sample_data = file.read(2000)  # Đọc một phần của file để không quá tải

sample_data


"Forestry exports up 47% in two months\nVietnam's forestry exports totaled US$2.68 billion in the first two months of 2024, up 47.4% year-on-year, according to the Department of Forestry under the Ministry of Agriculture and Rural Development.\nMarch 12, 2024 | 09:13 am GMT+7\nMinistry of Finance wants 50% tax on e-cigarette atomizer\nMinistry of Finance wants 50% tax on e-cigarette atomizer\nThe Ministry of Finance has maintained its proposal to impose a 50% Most-Favored Nation (MFN) tariff on electronic atomizer, the device used to heat up liquid solution used for vaping.\nMarch 12, 2024 | 09:13 am GMT+7\nPM urges New Zealand firms to boost investment in Vietnam\nPM urges New Zealand firms to boost investment in Vietnam\nPrime Minister Pham Minh Chinh urged New Zealand companies to enhance their investment in Vietnam, focusing on innovation, education, and energy sectors.\nMarch 11, 2024 | 11:00 pm GMT+7\nVietnam goes past Chile to become second biggest fruit, vegetable exporter to C

In [8]:
import re

# Mở và đọc toàn bộ nội dung file
with open(file_path, 'r', encoding='utf-8') as file:
    full_data = file.read()

# Sử dụng biểu thức chính quy để tìm tất cả các tiêu đề lặp lại, đây sẽ là điểm phân cách giữa các bài báo
# Giả định rằng mỗi tiêu đề đều theo sau bởi một dòng trống hoặc thời gian (để đối phó với trường hợp tiêu đề cuối cùng)
split_pattern = r'(\n[A-Za-z].*\n)\1'

# Phân tách dữ liệu sử dụng tiêu đề lặp lại như điểm phân cách
articles = re.split(split_pattern, full_data)

# Loại bỏ các phần tử trống và kết quả là mỗi phần tử sẽ chứa tiêu đề và nội dung còn lại của bài báo
cleaned_articles = [articles[i] + articles[i + 1] for i in range(0, len(articles) - 1, 3)]

# Kiểm tra cấu trúc của một vài bài báo sau khi đã xử lý
for article in cleaned_articles[:5]:
    print(article[:500], '...\n---\n')

# Chúng ta sẽ kiểm tra cấu trúc của một vài bài báo sau khi xử lý để đảm bảo rằng tiền xử lý diễn ra như mong đợi.


In [9]:
# Cố gắng tìm và loại bỏ các tiêu đề lặp lại một cách đơn giản hơn
# Đầu tiên, tách dữ liệu dựa trên dấu xuống dòng để xem xét từng dòng một cách riêng biệt
lines = full_data.split('\n')

# Khởi tạo danh sách mới cho dữ liệu đã xử lý
processed_data = []
skip_next = False

for i in range(len(lines) - 1):
    if skip_next:
        # Bỏ qua dòng này vì nó là một phần của tiêu đề lặp lại
        skip_next = False
        continue
    # Kiểm tra nếu dòng hiện tại và dòng tiếp theo giống nhau (tiêu đề lặp lại)
    if lines[i] == lines[i + 1]:
        processed_data.append('Title: ' + lines[i])  # Thêm tiêu đề một lần
        skip_next = True  # Đánh dấu để bỏ qua dòng tiếp theo vì nó là một bản sao
    else:
        processed_data.append(lines[i])

# Đối với mỗi bài báo, giờ đây chúng ta cần định dạng lại để thêm các nhãn cho mô tả và thời gian
# Ví dụ, chúng ta có thể tìm kiếm mẫu thời gian để phân biệt các phần

# Hãy xem qua dữ liệu đã xử lý:
for line in processed_data[:30]:  # Hiển thị một số dòng đầu tiên
    print(line)

# Lưu ý: Các bước tiếp theo như việc đánh dấu mô tả và thời gian cần thông tin thêm từ bạn để hoàn thiện


Forestry exports up 47% in two months
Vietnam's forestry exports totaled US$2.68 billion in the first two months of 2024, up 47.4% year-on-year, according to the Department of Forestry under the Ministry of Agriculture and Rural Development.
March 12, 2024 | 09:13 am GMT+7
Title: Ministry of Finance wants 50% tax on e-cigarette atomizer
The Ministry of Finance has maintained its proposal to impose a 50% Most-Favored Nation (MFN) tariff on electronic atomizer, the device used to heat up liquid solution used for vaping.
March 12, 2024 | 09:13 am GMT+7
Title: PM urges New Zealand firms to boost investment in Vietnam
Prime Minister Pham Minh Chinh urged New Zealand companies to enhance their investment in Vietnam, focusing on innovation, education, and energy sectors.
March 11, 2024 | 11:00 pm GMT+7
Title: Vietnam goes past Chile to become second biggest fruit, vegetable exporter to China
Vietnam surpassed Chile to become the second largest fruit and vegetable exporter to China in 2023, th

In [10]:
# Sử dụng biểu thức chính quy để tìm kiếm và đánh dấu thời gian trong từng bài báo
time_pattern = r'(\bJanuary|\bFebruary|\bMarch|\bApril|\bMay|\bJune|\bJuly|\bAugust|\bSeptember|\bOctober|\bNovember|\bDecember)\s\d{1,2},\s\d{4}\s\|\s\d{2}:\d{2}\s(am|pm)\sGMT\+7'

# Khởi tạo danh sách mới cho dữ liệu đã được đánh dấu thêm
marked_data = []

for line in processed_data:
    # Tìm và đánh dấu thời gian
    if re.search(time_pattern, line):
        marked_line = re.sub(time_pattern, r'Time: \g<0>', line)
        marked_data.append(marked_line)
    else:
        marked_data.append(line)

# Để đánh dấu mô tả, chúng ta cần xác định vị trí của nó giữa tiêu đề và thời gian
# Chúng ta sẽ làm điều này bằng cách tạo một cấu trúc mới cho mỗi bài báo, nơi mỗi bài báo được chứa trong một dictionary
final_articles = []
article = {"Title": "", "Description": "", "Time": ""}

for line in marked_data:
    if line.startswith("Title: "):
        # Nếu đây là bắt đầu của một bài báo mới và bài trước đó đã được xử lý, thêm nó vào danh sách
        if article["Title"]:  # Kiểm tra xem đã có tiêu đề chưa
            final_articles.append(article)
            article = {"Title": "", "Description": "", "Time": ""}
        article["Title"] = line[len("Title: "):]
    elif line.startswith("Time: "):
        article["Time"] = line[len("Time: "):]
    else:
        # Mô tả sẽ là phần còn lại giữa tiêu đề và thời gian
        article["Description"] += line + "\n"

# Đừng quên thêm bài viết cuối cùng vào danh sách sau khi thoát khỏi vòng lặp
if article["Title"]:
    final_articles.append(article)

# Hiển thị cấu trúc của một vài bài báo đầu tiên để kiểm tra
for article in final_articles[:5]:
    print(f"Title: {article['Title']}\nDescription: {article['Description'][:100]}... \nTime: {article['Time']}\n---\n")


Title: Ministry of Finance wants 50% tax on e-cigarette atomizer
Description: Forestry exports up 47% in two months
Vietnam's forestry exports totaled US$2.68 billion in the firs... 
Time: March 12, 2024 | 09:13 am GMT+7
---

Title: PM urges New Zealand firms to boost investment in Vietnam
Description: Prime Minister Pham Minh Chinh urged New Zealand companies to enhance their investment in Vietnam, f... 
Time: March 11, 2024 | 11:00 pm GMT+7
---

Title: Vietnam goes past Chile to become second biggest fruit, vegetable exporter to China
Description: Vietnam surpassed Chile to become the second largest fruit and vegetable exporter to China in 2023, ... 
Time: March 11, 2024 | 10:06 am GMT+7
---

Title: Chỉ còn 200 suất sở hữu vật phẩm giải chạy Việt Nam Hùng Cường
Description: Quảng cáo

... 
Time: 
---

Title: Vietnamese tech talents find homeland has appealing career prospects
Description: Vietnamese tech workers studying and working abroad are returning home in increasing numbers sin

In [11]:
import pandas as pd

# Chuyển đổi dữ liệu cuối cùng thành DataFrame
df_final_articles = pd.DataFrame(final_articles)

# Lưu DataFrame vào file CSV
csv_file_path = 'vn_economic_newspaper_from_vnexpress.csv'
df_final_articles.to_csv(csv_file_path, index=False, encoding='utf-8')

print(f"Dữ liệu đã được lưu thành công vào {csv_file_path}")


Dữ liệu đã được lưu thành công vào vn_economic_newspaper_from_vnexpress.csv


In [12]:
data = pd.read_csv('vn_economic_newspaper_from_vnexpress.csv')

# Tách phần ngày tháng ra từ cột 'Time', loại bỏ thông tin về giờ và múi giờ
data['Time'] = data['Time'].str.split('|').str[0].str.strip()

# Chuyển đổi chuỗi ngày tháng thành đối tượng datetime
data['Time'] = pd.to_datetime(data['Time'], format='%B %d, %Y', errors='coerce')

# Định dạng lại ngày tháng về dạng YYYY-MM-DD
data['Time'] = data['Time'].dt.date

# Hiển thị một số hàng đầu tiên để kiểm tra
data.head()


,Title,Description,Time
0,Ministry of Finance wants 50% tax on e-cigaret...,Forestry exports up 47% in two months\nVietnam...,2024-03-12
1,PM urges New Zealand firms to boost investment...,Prime Minister Pham Minh Chinh urged New Zeala...,2024-03-11
2,Vietnam goes past Chile to become second bigge...,Vietnam surpassed Chile to become the second l...,2024-03-11
3,Chỉ còn 200 suất sở hữu vật phẩm giải chạy Việ...,Quảng cáo\n\n,NaT
4,Vietnamese tech talents find homeland has appe...,Vietnamese tech workers studying and working a...,2024-03-11


In [13]:
# Save the preprocessed data to a new CSV file
new_file_path = 'vn_economic_newspaper_from_vnexpress_preprocessed.csv'
data.to_csv(new_file_path, index=False)

# Provide the path to the new file
new_file_path


'vn_economic_newspaper_from_vnexpress_preprocessed.csv'

In [14]:
import pandas as pd
from datetime import datetime

# Đọc nội dung tệp
file_path = 'vietnamtimes_economic_paper.txt'
with open(file_path, 'r', encoding='utf-8') as file:
    content = file.readlines()

# Hàm xử lý nội dung và trích xuất thông tin mong muốn
def parse_content_fixed_date(lines):
    articles = []
    i = 0
    while i < len(lines):
        # Bỏ qua tiêu đề bị lặp
        if i + 1 < len(lines) and lines[i] == lines[i + 1]:
            i += 1
        # Trích xuất ngày tháng và chuyển đổi thành đối tượng datetime
        elif 'Economy |' in lines[i] or 'Focus |' in lines[i] or 'Make in Vietnam |' in lines[i]:
            try:
                date_str = lines[i].split('|')[1].strip()
                date_obj = datetime.strptime(date_str, "%B %d, %Y")
            except ValueError:
                date_obj = datetime(1900, 1, 1)
            title = lines[i-1].strip()
            description = lines[i+1].strip() if i + 1 < len(lines) else ""
            articles.append({
                "Title": title,
                "Description": description,
                "Time": date_obj
            })
            i += 2
        else:
            i += 1
    return articles

# Xử lý nội dung và tạo DataFrame
articles_data_fixed = parse_content_fixed_date(content)
df_articles_fixed = pd.DataFrame(articles_data_fixed)

# Lưu DataFrame vào tệp CSV
output_file_path = 'vn_economic_newspaper_from_vietnamtimes.csv'
df_articles_fixed.to_csv(output_file_path, index=False)


In [15]:
import pandas as pd

preprocessed_data_path = 'vn_economic_newspaper_from_vnexpress_preprocessed.csv'
updated_data_path = 'vn_economic_newspaper_from_vietnamtimes.csv'

# Nạp bộ dữ liệu đã được tiền xử lý
preprocessed_data = pd.read_csv(preprocessed_data_path)
# Nạp bộ dữ liệu mới được cập nhật
updated_data = pd.read_csv(updated_data_path)

# Đảm bảo tính nhất quán về tên cột cho 'Time'
if 'Date' in updated_data.columns:
    updated_data.rename(columns={'Date': 'Time'}, inplace=True)

# Gộp hai bộ dữ liệu
merged_data = pd.concat([preprocessed_data, updated_data], ignore_index=True)

# Lưu bộ dữ liệu đã gộp vào một tệp mới
merged_data_path = 'merged_vn_economic_paper_data.csv'
merged_data.to_csv(merged_data_path, index=False)

print(f"Bộ dữ liệu đã gộp được lưu tại: {merged_data_path}")


Bộ dữ liệu đã gộp được lưu tại: merged_vn_economic_paper_data.csv


In [16]:
import pandas as pd
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from textblob import TextBlob
from transformers import AutoModelForSequenceClassification, AutoTokenizer
import torch

# Đảm bảo rằng lexicon của VADER đã được tải xuống
nltk.download('vader_lexicon')

# Đọc dữ liệu
combined_df = pd.read_csv('merged_vn_economic_paper_data.csv')

# Khởi tạo SentimentIntensityAnalyzer cho VADER
sid = SentimentIntensityAnalyzer()

# Áp dụng hàm phân tích cảm xúc VADER
combined_df['Vader_Sentiment'] = combined_df['Description'].apply(lambda text: sid.polarity_scores(text)['compound'])

# Áp dụng hàm phân tích cảm xúc TextBlob
combined_df['TextBlob_Sentiment'] = combined_df['Description'].apply(lambda text: TextBlob(text).sentiment.polarity)

# Khởi tạo tokenizer và mô hình cho FinBERT
tokenizer = AutoTokenizer.from_pretrained('yiyanghkust/finbert-tone')
model = AutoModelForSequenceClassification.from_pretrained('yiyanghkust/finbert-tone')

def finbert_sentiment(text):
    inputs = tokenizer(text, return_tensors='pt', max_length=512, truncation=True)
    with torch.no_grad():
        logits = model(**inputs).logits
    sentiment = logits.softmax(dim=1).argmax().item()
    # Chuyển đổi số nguyên sang nhãn cảm xúc: 0 - negative, 1 - neutral, 2 - positive
    return ['negative', 'neutral', 'positive'][sentiment]

# Áp dụng hàm phân tích cảm xúc FinBERT
combined_df['FinBERT_Sentiment'] = combined_df['Description'].apply(finbert_sentiment)

# Lưu DataFrame đã cập nhật vào một file CSV mới
updated_csv_path = 'vn_economic_articles_all_sentiments.csv'
combined_df.to_csv(updated_csv_path, index=False)

print(f"Kết quả phân tích cảm xúc đã được lưu vào {updated_csv_path}")


/Users/ronny/Project test/finanl_predict 5/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
[nltk_data] Error loading vader_lexicon: <urlopen error [SSL:
[nltk_data]     CERTIFICATE_VERIFY_FAILED] certificate verify failed:
[nltk_data]     unable to get local issuer certificate (_ssl.c:1000)>


Kết quả phân tích cảm xúc đã được lưu vào vn_economic_articles_all_sentiments.csv


In [17]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler

# Định nghĩa đường dẫn cho các tệp
file_paths = {
    'e1vfvn30': 'E1VFVN30 ETF Stock Price History.csv',
    'fueip100': 'FUEIP100 ETF Stock Price History.csv',
    'fuemav30': 'FUEMAV30 ETF Stock Price History.csv',
    'fuessv30': 'FUESSV30 ETF Stock Price History.csv',
    'fuessv50': 'FUESSV50 ETF Stock Price History.csv',
    'fuessvfl': 'FUESSVFL ETF Stock Price History.csv',
    'fuevfvd': 'FUEVFVND ETF Stock Price History.csv',
    'fuevn100': 'FUEVN100 ETF Stock Price History.csv',
    'sentiment_analysis': 'vn_economic_articles_all_sentiments.csv'
}

# Tải và xử lý dữ liệu cổ phiếu
data_frames = {}
for symbol, path in file_paths.items():
    df = pd.read_csv(path)
    if symbol != 'sentiment_analysis':
        df['Date'] = pd.to_datetime(df['Date'])
        df['Symbol'] = symbol.upper()
        price_columns = ['Price', 'Open', 'High', 'Low']
        for column in price_columns:
            if column in df.columns:
                df[column] = df[column].str.replace(',', '').astype(float)
        scaler = MinMaxScaler(feature_range=(0, 100))
        df[price_columns] = scaler.fit_transform(df[price_columns])
    data_frames[symbol] = df

# Xử lý dữ liệu phân tích tình cảm
sentiment_data = data_frames['sentiment_analysis']
sentiment_data['Time'] = pd.to_datetime(sentiment_data['Time'])
daily_sentiment = sentiment_data.groupby('Time').agg({
    'Vader_Sentiment': 'mean',
    'TextBlob_Sentiment': 'mean',
    'FinBERT_Sentiment': lambda x: x.mode().iloc[0]
}).reset_index()
daily_sentiment.rename(columns={'Time': 'Date'}, inplace=True)

# Kết hợp tất cả dữ liệu cổ phiếu và phân tích tình cảm
combined_stock_data = pd.concat([df for symbol, df in data_frames.items() if symbol != 'sentiment_analysis'])
final_combined_data = pd.merge(combined_stock_data, daily_sentiment, on='Date', how='inner')

# Lưu dữ liệu đã xử lý vào một tệp CSV
processed_csv_path = 'processed_financial_sentiment_data_combined.csv'
final_combined_data.to_csv(processed_csv_path, index=False)

# Đường dẫn tệp CSV đã xử lý
processed_csv_path


'processed_financial_sentiment_data_combined.csv'